**Imports**

In [2]:
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch
import math
import re
from dataloader_sentences import SentencesDataset, get_batch


**Config Parameters**

In [3]:
# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

initializing..


**Load data and build dataloader**

In [4]:
# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
pth = 'training.txt'
sentences = open(pth).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
pth = 'vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)


loading text...


FileNotFoundError: [Errno 2] No such file or directory: 'training.txt'

**Define Model**

In [4]:
# =============================================================================
# Model
# =============================================================================
#init model
from transformer_model import BERT
print('initializing model...')
model = BERT(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)


initializing model...
initializing optimizer and loss...


**Train Model**

In [5]:
# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 10000
for it in range(n_iteration):

    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)

    #infer
    masked_input = batch['input']
    masked_target = batch['target']

    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)

    #compute the cross entropy loss
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)

    #compute gradients
    loss.backward()

    #apply gradients
    optimizer.step()

    #print step
    if it % print_each == 0:
        print('it:', it,
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))

    #reset gradients
    optimizer.zero_grad()



training...
it: 0  | loss 10.13  | Δw: 1.101
it: 10  | loss 9.57  | Δw: 0.47
it: 20  | loss 9.37  | Δw: 0.298
it: 30  | loss 9.19  | Δw: 0.234
it: 40  | loss 9.04  | Δw: 0.198
it: 50  | loss 8.87  | Δw: 0.179
it: 60  | loss 8.71  | Δw: 0.167
it: 70  | loss 8.58  | Δw: 0.147
it: 80  | loss 8.43  | Δw: 0.142
it: 90  | loss 8.23  | Δw: 0.129
it: 100  | loss 8.11  | Δw: 0.128
it: 110  | loss 7.91  | Δw: 0.127
it: 120  | loss 7.81  | Δw: 0.114
it: 130  | loss 7.62  | Δw: 0.111
it: 140  | loss 7.57  | Δw: 0.106
it: 150  | loss 7.44  | Δw: 0.098
it: 160  | loss 7.27  | Δw: 0.097
it: 170  | loss 7.23  | Δw: 0.09
it: 180  | loss 7.09  | Δw: 0.094
it: 190  | loss 7.02  | Δw: 0.089
it: 200  | loss 6.93  | Δw: 0.09
it: 210  | loss 6.88  | Δw: 0.086
it: 220  | loss 6.8  | Δw: 0.09
it: 230  | loss 6.73  | Δw: 0.083
it: 240  | loss 6.65  | Δw: 0.087
it: 250  | loss 6.68  | Δw: 0.084
it: 260  | loss 6.6  | Δw: 0.088
it: 270  | loss 6.51  | Δw: 0.085
it: 280  | loss 6.54  | Δw: 0.087
it: 290  | loss 6.

KeyboardInterrupt: 

**Results Analysis**

In [ ]:
# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )


print('end')
